<a href="https://colab.research.google.com/github/KyeoRaeHan/NLP/blob/main/pair_cls_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기

TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [ ]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

TPU 이외에 의존성 있는 패키지를 설치합니다.

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 693 kB/s 
     |████████████████████████████████| 806 kB 14.7 MB/s 
     |████████████████████████████████| 2.8 MB 58.4 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 397 kB 61.5 MB/s 
     |████████████████████████████████| 636 kB 47.5 MB/s 
     |████████████████████████████████| 829 kB 48.6 MB/s 
     |████████████████████████████████| 134 kB 58.8 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.4 MB/s 
     |████████████████████████████████| 1.1 MB 39.2 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 271 kB 63.7 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 
     |████████████████████████████████| 94 kB 693 kB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [4]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base", # 프리트레인 마친 언어모델의 이름
    downstream_task_name="pair-classification", # 다운스트림 태스크 이름
    downstream_corpus_name="klue-nli", # 다운스트림 데이터 이름
    downstream_model_dir="/content/drive/MyDrive/Do it_자연어처리/data", #파인튜닝된 모델의 체크포인트가 저장될 위치
    batch_size=32 if torch.cuda.is_available() else 4, #배치크기
    learning_rate=5e-5, #러닝레이트: 1회 step에 얼마나 업데이트할 지에 관한 크기
    max_seq_length=64, #토큰 기준 문장 최대 길이
    epochs=5, # 학습 에포크 수
    tpu_cores=0 if torch.cuda.is_available() else 8, # tpu 코어수
    seed=7, # 랜더시드, None을 입력하면 시드 고정 X
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [5]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [6]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/Do it_자연어처리/data', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


# 말뭉치 다운로드
실습에 사용할 말뭉치를 다운로드합니다.

In [7]:
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 45.3MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 35.5MB/s]


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name, #프리트레인 모델의 토크나이저 선언
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축
학습데이터를 만듭니다.

In [9]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset #데이터셋 역할, dataloader가 배치를 만들때 인스턴스를 제공
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KlueNLICorpus() #다운스트림태스크 데이터
train_dataset = ClassificationDataset(  #tokenizer를 활용해 모델이 학습할 수 있는 형태로 데이터를 가공
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader( #모델 학습을 위해 학습 데이터(train_dataset)을 배치 단위로 모델에 공급
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 11.972 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 100분간 잘껄 그래도 소닉붐땜에 2점준다 + 100분간 잤다.
INFO:ratsnlp:tokens: [CLS] 100 ##분간 잘 ##껄 그래도 소 ##닉 ##붐 ##땜에 2 ##점 ##준다 [SEP] 100 ##분간 잤 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [10]:
#input_ids: 인덱스로 변환된 토큰 시퀀스
#attention_mask: 해당 토큰이 패딩토큰인지 아닌지를 나타냄
#token_type_ids: 세그먼트 정보, Bert모델의 특징
#label: 정수로 바뀐 레이블 정보
train_dataset[0] 

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

In [11]:
train_dataset[0].token_type_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [12]:
train_dataset[0].label

1

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [13]:
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset), #샘플링 방식 정의
    collate_fn=nlpbook.data_collator, #샘플링방식을 적용해 뽑은 인스턴스를 배치로 만드는 역할, data_collator는 파이토치가 요구하는 자료형인 텐서로 바꾸는 역할
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.412 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 10명이 함께 사용하기 불편함없이 만족했다. + 10명이 함께 사용하기 불편함이 많았다.
INFO:ratsnlp:tokens: [CLS] 10명 ##이 함께 사용 ##하기 불편 ##함 ##없이 만족 ##했다 . [SEP] 10명 ##이 함께 사용 ##하기 불편 ##함이 많았 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 21000, 4017, 9158, 9021, 8268, 10588, 4421, 8281, 14184, 8258, 17, 3, 21000, 4017, 9158, 9021, 8268, 10588, 11467, 14338, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 쌍 분류를 수행할 모델을 초기화합니다.

In [14]:
from transformers import BertConfig, BertForSequenceClassification #프리트레인을 마친 모델 위에 파인튜닝(문서분류)한 태스크 모듈이 덧붙여진 형태의 모델 클래스
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [15]:
model = BertForSequenceClassification.from_pretrained( 
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 학습 준비
Task와 Trainer를 준비합니다.

In [16]:
from ratsnlp.nlpbook.classification import ClassificationTask #옵티마이저, 러닝레이트스케줄이 정의되어 있다.
task = ClassificationTask(model, args)

In [21]:
trainer = nlpbook.get_trainer(args) #파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU설정, 로그 및 체크포인트 등의 설정을 해준다

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-paircls`)에 저장됩니다.

In [22]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## 정리
1. 각종 설정 지정
2. 말뭉치를 내려받아 전처리
3. 프리트레인 모델을 전처리한 데이터로 파인튜닝